In [1]:
import numpy as np
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Flatten
from tensorflow.keras.models import Model

In [2]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

mid = MidiFile('pnowks/corner.mid') 
notes = []

for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

print(notes)

[36, 48, 55, 60, 55, 60, 62, 62, 64, 64, 55, 60, 55, 60, 64, 64, 62, 62, 53, 53, 60, 60, 62, 62, 65, 65, 57, 59, 57, 59, 65, 65, 64, 64, 55, 55, 60, 64, 60, 67, 64, 59, 67, 59, 60, 60, 67, 65, 67, 65, 57, 57, 60, 65, 60, 65, 69, 59, 69, 65, 59, 65, 69, 69, 67, 67, 36, 60, 48, 60, 64, 67, 64, 72, 67, 60, 72, 64, 64, 67, 67, 72, 72, 59, 74, 74, 64, 64, 67, 67, 74, 72, 74, 57, 59, 72, 64, 67, 64, 67, 72, 59, 71, 72, 57, 60, 71, 64, 67, 64, 67, 71, 60, 69, 71, 59, 69, 64, 64, 67, 67, 69, 67, 59, 69, 60, 67, 60, 60, 64, 67, 64, 59, 57, 65, 67, 65, 60, 64, 60, 64, 65, 64, 65, 55, 57, 64, 59, 59, 60, 60, 64, 62, 64, 53, 55, 62, 57, 57, 60, 62, 60, 62, 67, 53, 52, 67, 59, 59, 64, 67, 64, 69, 67, 57, 52, 60, 69, 64, 60, 69, 64, 69, 57, 55, 64, 64, 57, 60, 57, 60, 64, 62, 53, 64, 55, 62, 57, 57, 60, 62, 60, 53, 60, 52, 62, 60, 55, 57, 55, 57, 60, 50, 59, 52, 60, 59, 53, 53, 55, 55, 59, 59, 50, 41, 48, 50, 52, 53, 53, 57, 57, 60, 62, 60, 62, 64, 62, 52, 64, 60, 62, 50, 60, 57, 57, 53, 48, 53, 52,

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))
print(notes)

[array([0.34285714]), array([0.45714286]), array([0.52380952]), array([0.57142857]), array([0.52380952]), array([0.57142857]), array([0.59047619]), array([0.59047619]), array([0.60952381]), array([0.60952381]), array([0.52380952]), array([0.57142857]), array([0.52380952]), array([0.57142857]), array([0.60952381]), array([0.60952381]), array([0.59047619]), array([0.59047619]), array([0.5047619]), array([0.5047619]), array([0.57142857]), array([0.57142857]), array([0.59047619]), array([0.59047619]), array([0.61904762]), array([0.61904762]), array([0.54285714]), array([0.56190476]), array([0.54285714]), array([0.56190476]), array([0.61904762]), array([0.61904762]), array([0.60952381]), array([0.60952381]), array([0.52380952]), array([0.52380952]), array([0.57142857]), array([0.60952381]), array([0.57142857]), array([0.63809524]), array([0.60952381]), array([0.56190476]), array([0.63809524]), array([0.56190476]), array([0.57142857]), array([0.57142857]), array([0.63809524]), array([0.61904

In [4]:
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 100
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
X = np.array(X)
print(X.shape)
X[0, :, :]

(12004, 100, 1)


array([[0.34285714],
       [0.45714286],
       [0.52380952],
       [0.57142857],
       [0.52380952],
       [0.57142857],
       [0.59047619],
       [0.59047619],
       [0.60952381],
       [0.60952381],
       [0.52380952],
       [0.57142857],
       [0.52380952],
       [0.57142857],
       [0.60952381],
       [0.60952381],
       [0.59047619],
       [0.59047619],
       [0.5047619 ],
       [0.5047619 ],
       [0.57142857],
       [0.57142857],
       [0.59047619],
       [0.59047619],
       [0.61904762],
       [0.61904762],
       [0.54285714],
       [0.56190476],
       [0.54285714],
       [0.56190476],
       [0.61904762],
       [0.61904762],
       [0.60952381],
       [0.60952381],
       [0.52380952],
       [0.52380952],
       [0.57142857],
       [0.60952381],
       [0.57142857],
       [0.63809524],
       [0.60952381],
       [0.56190476],
       [0.63809524],
       [0.56190476],
       [0.57142857],
       [0.57142857],
       [0.63809524],
       [0.619

In [5]:
def generator():
    gen_input = Input(shape = (None, 100))
    X = LSTM(256, return_sequences = True)(gen_input)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = LSTM(64, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    out = Dense(1, activation = "relu")(X)
    model = Model(inputs = gen_input, outputs = out)
    return model

In [6]:
def discriminator():
    disc_input = Input(shape = (100, 1))
    X = LSTM(128, return_sequences = True)(disc_input)
    X = Dropout(0.3)(X)
    X = LSTM(64, return_sequences = True)(X)
    X = Dropout(0.3)(X)
    X = Flatten()(X)
    X = Dense(128, activation = "relu")(X)
    X = Dropout(0.3)(X)
    out = Dense(1, activation = "sigmoid")(X)
    model = Model(inputs = disc_input, outputs = out)
    return model

In [7]:
def disc_with_gen(generator, discriminator):
    inputs = Input(shape = (None, 100))
    generated = generator(inputs)
    outputs = discriminator(generated)
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [8]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(0.0002, 0.5)
d = discriminator()
d.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
g = generator()

In [9]:
d.trainable = False
combined = disc_with_gen(g, d)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [10]:
X.shape

(12004, 100, 1)

In [11]:
from matplotlib import pyplot as plt
assert not np.any(np.isnan(X))
#generated = g.predict(noise)
#generated

In [12]:
disc_loss = []
gen_loss = []

In [14]:
idx = list(range(0, 64, 1))
real_data = X[idx]
real_data.shape

(64, 100, 1)

In [15]:
def train(epochs, num_of_batches, batch_size = 64):
    for epoch in range(epochs):
        a = 0
        b = 64
        for num in range(num_of_batches):
            idx = list(range(a, b, 1))
            real_data = X[idx]

            noise = np.random.normal(0, 1, (64, 100, 100))

            generated = g.predict(noise)
            d_loss_real = d.train_on_batch(real_data, np.ones((64, 1)))
            d_loss_fake = d.train_on_batch(generated, np.zeros((64, 1)))
            d_loss = 0.5 * np.add(real_loss, fake_loss)
            
            noise = np.random.normal(0, 1, (64, 100, 100))
            g_loss = combined.train_on_batch(noise, np.ones((64, 1)))

            a = a + 64
            b = b + 64
            
        if epoch % 1 == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            disc_loss.append(d_loss[0])
            gen_loss.append(g_loss)
            
    plt.plot(disc_loss, c='red')
    plt.plot(gen_loss, c='blue')
    plt.title("GAN Loss per Epoch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
    plt.close()

In [31]:
train(epochs = 5, num_of_batches = 187)

0 [D loss: 0.694071, acc.: 50.00%] [G loss: 0.699262]
1 [D loss: 0.693166, acc.: 50.00%] [G loss: 0.694989]
2 [D loss: 0.693216, acc.: 49.22%] [G loss: 0.696492]
3 [D loss: 0.693169, acc.: 50.00%] [G loss: 0.696275]
4 [D loss: 0.693169, acc.: 50.00%] [G loss: 0.696294]


In [32]:
pred = g.predict(np.random.normal(0, 1, (1, 100, 100)))
d.predict(pred)

array([[0.49842876]], dtype=float32)

In [33]:
d.predict(X[100, :, :].reshape(1, 100, 1))

array([[0.49842876]], dtype=float32)

In [38]:
prediction = g.predict(np.random.normal(0, 1, (1, 100, 100)))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]
#prediction = np.argmax(prediction, -1)

In [39]:
print(prediction)
#prediction = np.squeeze(prediction)

[56, 101, 84, 55, 58, 107, 120, 71, 40, 45, 87, 121, 99, 55, 34, 54, 90, 83, 52, 33, 58, 84, 79, 48, 32, 71, 113, 106, 75, 72, 67, 73, 94, 93, 79, 55, 63, 104, 120, 88, 53, 54, 93, 118, 88, 48, 36, 50, 67, 64, 49, 44, 42, 41, 41, 46, 68, 90, 78, 57, 53, 64, 78, 74, 53, 36, 38, 51, 61, 69, 69, 82, 90, 89, 82, 85, 76, 78, 89, 98, 93, 79, 66, 69, 78, 93, 95, 96, 81, 69, 75, 95, 92, 64, 58, 78, 92, 80, 63, 61]


In [44]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
t = 230
for note in prediction:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    if t >= 160:
        t -= 2
    msg.time = int(t)
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')

In [46]:
g.save("Model1.h5")

In [47]:
g.save_weights("Model1_weights.h5")

In [ ]:
loaded = tf.keras.models.load_model("Model1.h5")

In [45]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

mid = MidiFile('LSTM_music.mid')
#mid = MidiFile('pnowks/corner.mid') 
notes = []

for msg in mid:
    print(msg)

note_on channel=3 note=56 velocity=67 time=0.2375
note_on channel=3 note=101 velocity=67 time=0.23541666666666666
note_on channel=3 note=84 velocity=67 time=0.23333333333333334
note_on channel=3 note=55 velocity=67 time=0.23124999999999998
note_on channel=3 note=58 velocity=67 time=0.22916666666666666
note_on channel=3 note=107 velocity=67 time=0.22708333333333333
note_on channel=3 note=120 velocity=67 time=0.225
note_on channel=3 note=71 velocity=67 time=0.22291666666666665
note_on channel=3 note=40 velocity=67 time=0.22083333333333333
note_on channel=3 note=45 velocity=67 time=0.21875
note_on channel=3 note=87 velocity=67 time=0.21666666666666667
note_on channel=3 note=121 velocity=67 time=0.21458333333333332
note_on channel=3 note=99 velocity=67 time=0.2125
note_on channel=3 note=55 velocity=67 time=0.21041666666666667
note_on channel=3 note=34 velocity=67 time=0.20833333333333334
note_on channel=3 note=54 velocity=67 time=0.20625
note_on channel=3 note=90 velocity=67 time=0.2041666

In [30]:
b = [1,2,3,4,5,6,7,8,9]
a = list(np.array(b).reshape(-1,1))
note = [list(v) for v in a]
print(a)

[array([1]), array([2]), array([3]), array([4]), array([5]), array([6]), array([7]), array([8]), array([9])]
